In [1]:
from SimPy.Simulation import *
import random
import numpy as np
import math
import pandas as pd
import scipy.stats as ss
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def conf(L):
    """confidence interval"""
    lower = np.mean(L) - 1.96*np.std(L)/math.sqrt(len(L))
    upper = np.mean(L) + 1.96*np.std(L)/math.sqrt(len(L))
    return (lower, upper)

In [3]:
data = pd.read_csv("Cafe Louis Data.csv").iloc[:,1:]
data.head()

,Inter-Arrival Time,Waiting Time,Service Time
0,0.00,0.00,51.90
1,15.38,43.96,35.75
2,68.47,0.00,25.13
3,13.09,0.00,33.16
4,19.40,11.14,31.19


In [4]:
print("Mean Interarrival Time: ", np.mean(data["Inter-Arrival Time"]))
print("Mean Service Time: ", np.mean(data["Service Time"]))

Mean Interarrival Time:  27.674940239043824
Mean Service Time:  41.30215139442231


In [5]:
exp_arrival_rate = 1/np.mean(data["Inter-Arrival Time"])
exp_service_rate = 1/np.mean(data["Service Time"])

In [6]:
print("Expected Arrival rate: %s, Expected Service Rate: %s, p : %s" %(exp_arrival_rate,exp_service_rate,exp_arrival_rate/exp_service_rate))

Expected Arrival rate: 0.03613377269697585, Expected Service Rate: 0.024211813821762466, p : 1.4924025503821399


## Model 1 (M/M/c)

In [7]:
## Model ----------
class Source(Process):
    """generate random arrivals"""
    def run(self, N, lamb, mu):
        for i in range(N):
            a = Arrival(str(i))
            activate(a, a.run(mu))
            t = random.expovariate(lamb)
            yield hold, self, t

class Arrival(Process):
    n = 0
    """an arrival"""
    def run(self, mu):
        Arrival.n += 1
        arrivetime = now()
        G.numbermon.observe(Arrival.n)
        yield request, self, G.server
        t = random.expovariate(mu)
        yield hold, self, t
        yield release, self, G.server
        Arrival.n -= 1
        G.numbermon.observe(Arrival.n)
        delay = now()-arrivetime
        G.delaymon.observe(delay)

class G:
    server = 'dummy'
    delaymon = 'Monitor'
    numbermon = 'Monitor'

def model(c, N, lamb, mu, maxtime, rvseed):
    # setup
    initialize()
    random.seed(rvseed)
    G.server = Resource(c)
    G.delaymon = Monitor()
    G.numbermon = Monitor()
   
    # simulate
    s = Source('Source')
    activate(s, s.run(N, lamb, mu))
    simulate(until=maxtime)
   
    # gather performance measures
    W = G.delaymon.mean()
    L = G.numbermon.timeAverage()
    Lambda_eff = L/W
    return W, L, Lambda_eff

In [8]:
all_W = []
all_L = []
all_Lambda_eff = []
for k in range(50):
    seed = 123*k
    W, L, Lambda_eff = model(1, 10000, exp_arrival_rate, exp_service_rate, 100000, seed)
    all_W.append(W)
    all_L.append(L)
    all_Lambda_eff.append(Lambda_eff)

In [9]:
print("Estimate of W: ", np.mean(all_W))
print("Estimate of L: ", np.mean(all_L))
print("Estimate of Lambda_eff: ", np.mean(all_Lambda_eff))

Estimate of W:  16519.214184910157
Estimate of L:  30063.44789093628
Estimate of Lambda_eff:  1.8413316657096608


## Model 2 (M/G/1)

Best Fit Distributions:
- Inter-Arrival Times: Exponential loc = 27.649
- Service Time: Erlang k = 2.12784 , loc= 5.3858, scale = 16.8792

In [10]:
# generating random erlang variates using estimated parameters

np.random.seed(123)

service = data["Service Time"]

fit_k,fit_loc,fit_scale = ss.erlang.fit(service)

erlang = ss.erlang(fit_k,fit_loc,fit_scale)

erlang.rvs()

C:\Users\kille\AppData\Roaming\Python\Python311\site-packages\scipy\stats\_continuous_distns.py:3180: RuntimeWarning: The shape parameter of the erlang distribution has been given a non-integer value 0.00025.
  warnings.warn(
C:\Users\kille\AppData\Roaming\Python\Python311\site-packages\scipy\stats\_continuous_distns.py:3180: RuntimeWarning: The shape parameter of the erlang distribution has been given a non-integer value 0.00016666666666666666.
  warnings.warn(
C:\Users\kille\AppData\Roaming\Python\Python311\site-packages\scipy\stats\_continuous_distns.py:3180: RuntimeWarning: The shape parameter of the erlang distribution has been given a non-integer value 0.0002777777777777778.
  warnings.warn(
C:\Users\kille\AppData\Roaming\Python\Python311\site-packages\scipy\stats\_continuous_distns.py:3180: RuntimeWarning: The shape parameter of the erlang distribution has been given a non-integer value 0.000462962962962963.
  warnings.warn(
C:\Users\kille\AppData\Roaming\Python\Python311\site-p

17.16237301242824

In [29]:
## Model ----------
class Source(Process):
    """generate random arrivals"""
    def run(self, N, lamb):
        for i in range(N):
            a = Arrival(str(i))
            activate(a, a.run())
            t = random.expovariate(lamb)
            yield hold, self, t

class Arrival(Process):
    n = 0
    """an arrival"""
    def run(self):
        Arrival.n += 1
        arrivetime = now()
        G.numbermon.observe(Arrival.n)
        yield request, self, G.server
        t = erlang.rvs() # generates random erlang variate for service time
        yield hold, self, t
        yield release, self, G.server
        Arrival.n -= 1
        G.numbermon.observe(Arrival.n)
        delay = now()-arrivetime
        G.delaymon.observe(delay)

class G:
    server = 'dummy'
    delaymon = 'Monitor'
    numbermon = 'Monitor'

def model(c, N, lamb, maxtime, rvseed):
    # setup
    initialize()
    random.seed(rvseed)
    np.random.seed(rvseed) # set random seed for scipy
    G.server = Resource(c)
    G.delaymon = Monitor()
    G.numbermon = Monitor()
   
    # simulate
    s = Source('Source')
    activate(s, s.run(N, lamb))
    simulate(until=maxtime)
   
    # gather performance measures
    W = G.delaymon
    L = G.numbermon.timeAverage()
    Lambda_eff = L/W
    return W, L, Lambda_eff

In [30]:
allW = []
allL = []
allLambda_eff = []
for k in range(50):
    seed = 123*k
    result_W, result_L, result_Lambda_eff = model(c=1, N=10000, lamb=exp_service_rate, maxtime=100000, rvseed=seed)
    allW.append(result_W)
    allL.append(result_L)
    allLambda_eff.append(result_Lambda_eff)

C:\Users\kille\AppData\Roaming\Python\Python311\site-packages\scipy\stats\_continuous_distns.py:3180: RuntimeWarning: The shape parameter of the erlang distribution has been given a non-integer value array(2.12783986).
  warnings.warn(


In [16]:
print("Estimate of W: ", np.mean(allW))
print("Estimate of L: ", np.mean(allL))
print("Estimate of Lambda_eff: ", np.mean(allLambda_eff))

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (50,) + inhomogeneous part.

## Model 3 ( Empirical Distribution)

In [21]:
def ecdf(data):
    n = len(data)
    x = np.sort(data)
    y = np.arange(1,n+1)/n
    return x, y

In [22]:
def draw_emp(data, r):
    x , y = ecdf(data)
    return np.interp(r ,y,x)

In [23]:
## Model ----------
class Source(Process):
    """generate random arrivals"""
    def run(self, N, lamb, mu):
        for i in range(N):
            a = Arrival(str(i))
            activate(a, a.run(mu))
            r = random.uniform(0,1)
            t = draw_emp(data['Inter-Arrival Time'], r) # generate random empirical variate for inter-arrival time from dist
            yield hold, self, t

class Arrival(Process):
    n = 0
    """an arrival"""
    def run(self, mu):
        Arrival.n += 1
        arrivetime = now()
        G.numbermon.observe(Arrival.n)
        yield request, self, G.server
        r = random.uniform(0,1)
        t = draw_emp(data["Service Time"], r) # generate random empirical variate for service time from dist
        yield hold, self, t
        yield release, self, G.server
        Arrival.n -= 1
        G.numbermon.observe(Arrival.n)
        delay = now()-arrivetime
        G.delaymon.observe(delay)

class G:
    server = 'dummy'
    delaymon = 'Monitor'
    numbermon = 'Monitor'

def model(c, N, lamb, mu, maxtime, rvseed):
    # setup
    initialize()
    random.seed(rvseed)
    G.server = Resource(c, monitored=True)
    G.delaymon = Monitor()
    G.numbermon = Monitor()
   
    # simulate
    s = Source('Source')
    activate(s, s.run(N, lamb, mu))
    simulate(until=maxtime)
   
    # gather performance measures
    W = G.delaymon.mean()
    # L
    L = G.numbermon.timeAverage()
    # Lambda_eff = l / w
    Lambda_eff = L / W
    
    
    return W, L, Lambda_eff



In [24]:
allW = []
allL = []
allLambda_eff = []
for k in range(50):
    seed = 123*k
    result_W, result_L, result_Lambda_eff = model(c=1,N=10000,lamb=exp_arrival_rate,mu=exp_service_rate,maxtime=10000000,rvseed=1234)
    allW.append(result_W)
    allL.append(result_L)
    allLambda_eff.append(result_Lambda_eff)

In [25]:
print("Estimate of W: ", np.mean(allW))

print("Estimate of L: ", np.mean(allL))

print("Estimate of Lambda_eff: ", np.mean(allLambda_eff))

Estimate of W:  67236.29270123183
Estimate of L:  1654.8244356116716
Estimate of Lambda_eff:  0.024612071384794744
